In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [16]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = [1, 2, 3, 4, 5, 6, 7, 8, 9]
EXP = "BAYESIAN_MODEL"


In [17]:
import os
def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [18]:
from SVM_models.QuantumSVM import QuantumSVM
from sklearn.metrics import accuracy_score

In [19]:
from bayes_opt import BayesianOptimization

I = np.array([[1, 0], [0, 1]])  # Identity
X = np.array([[0, 1], [1, 0]])  # Pauli-X
Y = np.array([[0, -1j], [1j, 0]])  # Pauli-Y
Z = np.array([[1, 0], [0, -1]]) 

L = 150

def generate_H_from_C(coefficients):
    """Generates a random Hamiltonian for the given number of qubits by combining Pauli matrices."""
    pauli_matrices = [I, X, Y, Z]  # Random coefficients for each term
    H = None
    # Generate all possible combinations of Pauli matrices for the given number of qubits
    for i in range(4**CONFIG["N_QUBITS"]):
        indices = np.unravel_index(i, (4,) * CONFIG["N_QUBITS"])
        kron_product = pauli_matrices[indices[0]]
        for idx in indices[1:]:
            kron_product = np.kron(kron_product, pauli_matrices[idx])
        kron_product = np.array(kron_product, dtype=complex)
        if H is None:
            H = coefficients[i] * kron_product
        else:
            H += coefficients[i] * kron_product
    H = H / np.linalg.norm(H)  # Normalize the Hamiltonian
    return H

def generate_random_alphas():
    return np.random.rand(L)

In [ ]:
for SEED in SEEDS:

    random.seed(SEED)

    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)
    
    def objective_function(**kwargs):
        # Convert kwargs to a coefficient array
        coefficients = np.array([kwargs[f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])])
        alphas_c = np.array([kwargs[f"w_{i}"] for i in range(L)])
        #print("Coefficients:", coefficients)
        # Fit the QuantumSVM model with the generated Hamiltonian
        quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=coefficients, alphas_c=alphas_c, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])
        quantum_svm.fit(train_features, train_labels)
        
        y_val = quantum_svm.predict(val_features)
        val_accuracy = accuracy_score(val_labels, y_val)
        return val_accuracy


    pbounds = {f"coeff_{i}": (0, 1) for i in range(4**CONFIG["N_QUBITS"])}
    pbounds.update({f"w_{i}": (0, 1) for i in range(L)})

    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds=pbounds,
        random_state=SEED,
        verbose=2
    )

    optimizer.maximize(
        init_points=5,  # Number of random initial points
        n_iter=OPT_TIMES-5        # Number of optimization steps
    )
    # Retrieve the best coefficients
    best_coefficients = [optimizer.max["params"][f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])]
    best_alphas = [optimizer.max["params"][f"w_{i}"] for i in range(L)]

    #convert best_alphas to numpy array
    best_alphas = np.array(best_alphas)

    print("Best Coefficients:", best_coefficients)
    print("Best Val Accuracy:", optimizer.max["target"])

    quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=best_coefficients, alphas_c=best_alphas, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])

    quantum_svm.fit(train_features, train_labels)
    y_train = quantum_svm.predict_on_train()
    train_accuracy = accuracy_score(train_labels, y_train)
    print(f"Train Accuracy: {train_accuracy}")

    y_val = quantum_svm.predict(val_features)
    val_accuracy = accuracy_score(val_labels, y_val)
    print(f"Validation Accuracy: {val_accuracy}")

    y_test = quantum_svm.predict(test_features)
    accuracy = accuracy_score(test_labels, y_test)
    print(f"Test Accuracy: {accuracy}")


    # now I want to save the model Hamiltonian_c and hidden_alphas
    print("Saving model...")
    Hamiltonian_c = quantum_svm.Hamiltonian_c
    hidden_alphas = quantum_svm.hidden_alphas

    #print("Hamiltonian_c: ", Hamiltonian_c)
    #print("hidden_alphas: ", hidden_alphas)

    save_results(EXP, CONFIG["ENCODING"], SEED, train_accuracy, val_accuracy, accuracy, Hamiltonian_c, hidden_alphas)

|   iter    |  target   |  coeff_0  |  coeff_1  | coeff_10  | coeff_11  | coeff_12  | coeff_13  | coeff_14  | coeff_15  |  coeff_2  |  coeff_3  |  coeff_4  |  coeff_5  |  coeff_6  |  coeff_7  |  coeff_8  |  coeff_9  |    w_0    |    w_1    |   w_10    |   w_100   |   w_101   |   w_102   |   w_103   |   w_104   |   w_105   |   w_106   |   w_107   |   w_108   |   w_109   |   w_11    |   w_110   |   w_111   |   w_112   |   w_113   |   w_114   |   w_115   |   w_116   |   w_117   |   w_118   |   w_119   |   w_12    |   w_120   |   w_121   |   w_122   |   w_123   |   w_124   |   w_125   |   w_126   |   w_127   |   w_128   |   w_129   |   w_13    |   w_130   |   w_131   |   w_132   |   w_133   |   w_134   |   w_135   |   w_136   |   w_137   |   w_138   |   w_139   |   w_14    |   w_140   |   w_141   |   w_142   |   w_143   |   w_144   |   w_145   |   w_146   |   w_147   |   w_148   |   w_149   |   w_15    |   w_16    |   w_17    |   w_18    |   w_19    |    w_2    |   w_20    |   w_21    |   

Building TRAIN_KERNEL:   0%|          | 0/1225 [00:00<?, ?it/s]

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:35<00:00, 26.18it/s]


| 1         | 0.48      | 0.417     | 0.7203    | 0.0001144 | 0.3023    | 0.1468    | 0.09234   | 0.1863    | 0.3456    | 0.3968    | 0.5388    | 0.4192    | 0.6852    | 0.2045    | 0.8781    | 0.02739   | 0.6705    | 0.4173    | 0.5587    | 0.1404    | 0.1981    | 0.8007    | 0.9683    | 0.3134    | 0.6923    | 0.8764    | 0.8946    | 0.08504   | 0.03905   | 0.1698    | 0.8781    | 0.09835   | 0.4211    | 0.9579    | 0.5332    | 0.6919    | 0.3155    | 0.6865    | 0.8346    | 0.01829   | 0.7501    | 0.9889    | 0.7482    | 0.2804    | 0.7893    | 0.1032    | 0.4479    | 0.9086    | 0.2936    | 0.2878    | 0.13      | 0.01937   | 0.6788    | 0.2116    | 0.2655    | 0.4916    | 0.05336   | 0.5741    | 0.1467    | 0.5893    | 0.6998    | 0.1023    | 0.4141    | 0.6944    | 0.4142    | 0.04995   | 0.5359    | 0.6638    | 0.5149    | 0.9446    | 0.5866    | 0.9034    | 0.1375    | 0.1393    | 0.8074    | 0.3977    | 0.1654    | 0.9275    | 0.3478    | 0.7508    | 0.726     | 0.8833    | 0.

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:09<00:00, 35.78it/s]


...

Building TEST_KERNEL: 100%|██████████| 2500/2500 [25:59<00:00,  1.60it/s]  


| 11        | 0.76      | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 1.

Building TEST_KERNEL: 100%|██████████| 2500/2500 [25:48<00:00,  1.61it/s]  


| 12        | 0.82      | 0.0       | 0.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 0.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 1.0       | 0.0       | 1.0       | 0.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 1.0       | 0.0       | 0.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 1.0       | 0.0       | 1.

Building TEST_KERNEL:  79%|███████▉  | 1978/2500 [22:16<08:04,  1.08it/s]  